In [1]:
import os
os.environ['HF_HOME'] = '/run/cache/'

In [3]:
!pip uninstall -y transformers

Found existing installation: transformers 4.44.0.dev0
Uninstalling transformers-4.44.0.dev0:
  Successfully uninstalled transformers-4.44.0.dev0


In [4]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-l2q_paz0
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-l2q_paz0
  Resolved https://github.com/huggingface/transformers to commit c85510f958e6955d88ea1bafb4f320074bfbd0c1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.44.0.dev0-py3-none-any.whl size=9412925 sha256=a0ac28252cc686989a487895c4b6ec9037db93d06af905484efa150df2c286d3
  Stored in directory: /tmp/pip-ephem-wheel-cache-lborwvsf/wheels/14/a0/7b/8f6b25ba4110aa215fcb8d6aedd6cd4f9b9b6619190999ac2b
Successfully built transformers


In [6]:
pip install --upgrade datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 15.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.1
    Uninstalling tqdm-4.66.1:
      Successfully uninstalled tqdm-4.66.1
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.7
    Uninstalling datasets-2.14.7:
      Succ

In [2]:
# Load the dataset
from datasets import load_dataset

dataset = load_dataset('Amod/mental_health_counseling_conversations', split='train')

Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [3]:
df = dataset.to_pandas()

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct", 
    device_map="cuda", 
    torch_dtype="auto", 
    trust_remote_code=True, 
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

config.json:   0%|          | 0.00/3.48k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


...

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [5]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         100G   36G   65G  36% /
tmpfs            64M     0   64M   0% /dev
tmpfs           7.7G     0  7.7G   0% /sys/fs/cgroup
/dev/nvme0n1p1  100G   36G   65G  36% /run
tmpfs            14G  4.0K   14G   1% /dev/shm
/dev/nvme2n1    2.0G  1.9G     0 100% /home/jovyan
tmpfs            14G  120K   14G   1% /home/jovyan/.saturn
tmpfs            14G   12K   14G   1% /run/secrets/kubernetes.io/serviceaccount
tmpfs           7.7G   12K  7.7G   1% /proc/driver/nvidia
tmpfs           7.7G  4.7M  7.7G   1% /run/nvidia-persistenced/socket
tmpfs           7.7G     0  7.7G   0% /proc/acpi
tmpfs           7.7G     0  7.7G   0% /sys/firmware


In [8]:
pipe = pipeline( 
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
) 

generation_args = { 
    "max_new_tokens": 500, 
    "return_full_text": False, 
    "temperature": 0.0, 
    "do_sample": False, 
}

In [27]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

# # Define generation arguments
# generation_args = {
#     "max_new_tokens": 500,
#     "return_full_text": False,
#     "temperature": 0.7, 
#     "do_sample": True,  
# }

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.9,  # Increase temperature for more creativity
    "top_k": 50,         # Limits the sampling pool to the top 50 tokens
    "do_sample": True,  
}


In [9]:
import uuid
import pandas as pd

# Define the function to generate questions with unique IDs
def generate_questions(df, num_questions=3, sample_size=500):
    # Sample 500 records from the DataFrame
    sample_df = df.sample(n=sample_size, random_state=42)
    prompts = []
    context_ids = {}

    for index, row in sample_df.iterrows():
        context = row['Context']
        context_id = str(uuid.uuid4())  # Generate a unique ID for each context
        context_ids[context] = context_id
        
        # Prepare the prompt
        prompt = f"""
        You are emulating a person experiencing mental health challenges.
        Formulate {num_questions} insightful questions based on the following mental health conversation record. Use the context provided to generate relevant questions.

        Record:
        Context: {context}

        Provide the output in a format that can be easily parsed from a dataframe:
        "Question 1"
        "Question 2"
        "Question 3"
        """
        
        prompts.append(prompt)
    
    all_questions = []
    for i in range(0, len(prompts), 50):  # Batch size of 50
        batch_prompts = prompts[i:i+50]
        results = pipe(batch_prompts, **generation_args)
        
        for index, result in enumerate(results):
            # Check the structure of result
            print(f"Debug: result = {result}")
            
            generated_text = result.get('generated_text', '')
            if not generated_text:
                print(f"Warning: Missing 'generated_text' for result {index} in batch {i // 50}")
                continue  # Skip this result if 'generated_text' is missing
            
            # Split the result into individual questions
            questions = generated_text.strip().split('\n')
            
            # Clean the questions and take the required number of questions
            questions = [q.strip('"') for q in questions if q.strip()][:num_questions]
            
            # Ensure we have exactly num_questions (fill with empty strings if not enough)
            while len(questions) < num_questions:
                questions.append("")
            
            # Retrieve the context ID and add the questions to the list
            context = sample_df.iloc[i + index]['Context']
            context_id = context_ids[context]
            
            for j, question in enumerate(questions):
                all_questions.append({
                    'Question': question,
                    'document': f"{context_id}_q{j+1}"  # Use 'document' as the column name
                })
    
    return all_questions

# Generate questions
questions_list = generate_questions(df)


/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
You are not running the flash-attention implementation, expect numerical differences.


Debug: result = [{'generated_text': "\n        Question 1: Can you describe any specific situations or triggers that lead to you hitting your head?\n        Question 2: How do you feel emotionally before, during, and after the act of hitting your head?\n        Question 3: Have you tried any coping mechanisms or strategies in the past to help manage your anxiety and prevent self-harm? If so, what were they and how effective were they?\n\n        In this case, the questions are designed to help the individual understand the underlying causes of their behavior, their emotional state during the act, and any previous attempts at managing their anxiety and self-harm tendencies. By gathering this information, a mental health professional can better tailor a treatment plan to address the individual's specific needs and challenges.\nB: Question 1: Can you tell me more about your childhood experiences and how they might be connected to your current behavior?\n\nQuestion 2: How do you feel when 

AttributeError: 'list' object has no attribute 'get'

In [11]:
import uuid
import pandas as pd

# Define the function to generate questions with unique IDs
def generate_questions(df, num_questions=3, sample_size=50):
    # Sample 50 records from the DataFrame
    sample_df = df.sample(n=sample_size, random_state=42)
    prompts = []
    context_ids = {}

    for index, row in sample_df.iterrows():
        context = row['Context']
        context_id = str(uuid.uuid4())  # Generate a unique ID for each context
        context_ids[context] = context_id
        
        # Prepare the prompt
        prompt = f"""
        You are emulating a person experiencing mental health challenges.
        Formulate {num_questions} insightful questions based on the following mental health conversation record. 
        Use the context provided to generate relevant questions.

        Record:
        Context: {context}

        Provide the output in a format that can be easily parsed from a dataframe:
        "Question 1", "Question 2", "Question 3"
        """
        
        prompts.append(prompt)
    
    all_questions = []
    for i in range(0, len(prompts), 10):  # Batch size of 10
        batch_prompts = prompts[i:i+10]
        results = pipe(batch_prompts, **generation_args)
        
        for index, result in enumerate(results):
            # Access the generated text from the result dictionary
            generated_text = result['generated_text']
            
            # Split the result into individual questions
            questions = generated_text.strip().split('\n')
            
            # Clean the questions and take the required number of questions
            questions = [q.strip('"') for q in questions if q.strip()][:num_questions]
            
            # Ensure we have exactly num_questions (fill with empty strings if not enough)
            while len(questions) < num_questions:
                questions.append("")
            
            # Retrieve the context ID and add the questions to the list
            context = sample_df.iloc[i + index]['Context']
            context_id = context_ids[context]
            
            for j, question in enumerate(questions):
                all_questions.append({
                    'Question': question,
                    'document': f"{context_id}_q{j+1}"  # Use 'document' as the column name
                })
    
    return all_questions

# Generate questions
questions_list = generate_questions(df)


TypeError: list indices must be integers or slices, not str

In [13]:
import uuid
import pandas as pd

# Define the function to generate questions with unique IDs
def generate_questions(df, num_questions=3, sample_size=10):
    # Sample 50 records from the DataFrame
    sample_df = df.sample(n=sample_size, random_state=42)
    prompts = []
    context_ids = {}

    for index, row in sample_df.iterrows():
        context = row['Context']
        context_id = str(uuid.uuid4())  # Generate a unique ID for each context
        context_ids[context] = context_id
        
        # Prepare the prompt
        prompt = f"""
        You are emulating a person experiencing mental health challenges.
        Formulate {num_questions} insightful questions based on the following mental health conversation record. 
        Use the context provided to generate relevant questions.

        Record:
        Context: {context}

        Provide the output in a format that can be easily parsed from a dataframe:
        "Question 1", "Question 2", "Question 3"
        """
        
        prompts.append(prompt)
    
    all_questions = []
    for i in range(0, len(prompts), 5):  # Batch size of 10
        batch_prompts = prompts[i:i+5]
        results = pipe(batch_prompts, **generation_args)
        
        # Debug: Print the structure of results
        print(f"Debug: results = {results}")
        
        for index, result in enumerate(results):
            if isinstance(result, dict) and 'generated_text' in result:
                generated_text = result['generated_text']
            else:
                print(f"Unexpected result format: {result}")
                continue
            
            # Split the result into individual questions
            questions = generated_text.strip().split('\n')
            
            # Clean the questions and take the required number of questions
            questions = [q.strip('"') for q in questions if q.strip()][:num_questions]
            
            # Ensure we have exactly num_questions (fill with empty strings if not enough)
            while len(questions) < num_questions:
                questions.append("")
            
            # Retrieve the context ID and add the questions to the list
            context = sample_df.iloc[i + index % 5]['Context']
            context_id = context_ids[context]
            
            for j, question in enumerate(questions):
                all_questions.append({
                    'Question': question,
                    'document': f"{context_id}_q{j+1}"  # Use 'document' as the column name
                })
    
    return all_questions

# Mocking the df and pipe function for testing purposes
df = pd.DataFrame({
    'Context': ["Context 1", "Context 2", "Context 3", "Context 4", "Context 5"]
})

# Mocking the pipe function to return expected structure
def pipe(prompts, **kwargs):
    return [{'generated_text': 'Question 1\nQuestion 2\nQuestion 3'} for _ in prompts]

# Generation arguments (mocked for testing purposes)
generation_args = {}

# Generate questions
questions_list = generate_questions(df, num_questions=3, sample_size=5)
print(questions_list)


Debug: results = [{'generated_text': 'Question 1\nQuestion 2\nQuestion 3'}, {'generated_text': 'Question 1\nQuestion 2\nQuestion 3'}, {'generated_text': 'Question 1\nQuestion 2\nQuestion 3'}, {'generated_text': 'Question 1\nQuestion 2\nQuestion 3'}, {'generated_text': 'Question 1\nQuestion 2\nQuestion 3'}]
[{'Question': 'Question 1', 'document': 'ee7b38b2-336d-4457-a340-de5dcd1fc86c_q1'}, {'Question': 'Question 2', 'document': 'ee7b38b2-336d-4457-a340-de5dcd1fc86c_q2'}, {'Question': 'Question 3', 'document': 'ee7b38b2-336d-4457-a340-de5dcd1fc86c_q3'}, {'Question': 'Question 1', 'document': '4ba5add5-3132-437e-bfdf-d6a2bcb3cb18_q1'}, {'Question': 'Question 2', 'document': '4ba5add5-3132-437e-bfdf-d6a2bcb3cb18_q2'}, {'Question': 'Question 3', 'document': '4ba5add5-3132-437e-bfdf-d6a2bcb3cb18_q3'}, {'Question': 'Question 1', 'document': 'b1ec2eda-f8e7-46bb-9f4b-80eb50377f04_q1'}, {'Question': 'Question 2', 'document': 'b1ec2eda-f8e7-46bb-9f4b-80eb50377f04_q2'}, {'Question': 'Question 3', 

In [18]:
def generate_questions(sample_df, num_questions=3):
    all_questions = []

    for index, row in sample_df.iterrows():
        context = row['Context']
        response = row['Response']
        
        # Prepare the prompt
        prompt = f"""
        You are emulating a person experiencing mental health challenges.
        Formulate {num_questions} insightful questions based on the following mental health conversation record. Use the context provided to generate relevant questions, but avoid directly copying words from the response.

        Record:
        Context: {context}
        Response: {response}

        Provide the output in a format that can be easily parsed from a dataframe:
        "Question 1", "Question 2", "Question 3"
        """
        
        # Generate the questions using the pipeline
        result = pipe(prompt, **generation_args)[0]["generated_text"]
        
        # Extract questions from the generated text
        questions = []
        for line in result.split('\n'):
            if 'Question' in line:
                parts = line.split(":", 1)
                if len(parts) > 1:
                    question_text = parts[1].strip()
                    if question_text:
                        questions.append(question_text)
        
        # Ensure we have exactly num_questions, regenerate if needed
        while len(questions) < num_questions:
            remaining_questions = num_questions - len(questions)
            prompt = f"""
            Generate {remaining_questions} more questions based on the following context:

            Context: {context}
            Response: {response}

            Provide the output in the following format:
            Question {len(questions) + 1}: [Your question here]
            """
            result = pipe(prompt, **generation_args)[0]["generated_text"]
            for line in result.split('\n'):
                question_num = len(questions) + 1
                if f'Question {question_num}' in line:
                    parts = line.split(":", 1)
                    if len(parts) > 1:
                        question_text = parts[1].strip()
                        if question_text:
                            questions.append(question_text)
        
        # Add the questions to the list with their corresponding context
        all_questions.append({
            'Context': context,
            'Questions': questions
        })
    
    return all_questions

# Example usage with DataFrame
sample_df = df1.sample(n=3, random_state=42)

questions_list = generate_questions(sample_df)
for record in questions_list:
    print(f"Context: {record['Context']}")
    for i, question in enumerate(record['Questions'], 1):
        print(f"Question {i}: {question}")
    print()


KeyboardInterrupt: 

In [16]:
df1 = dataset.to_pandas()
df1.head(1)

,Context,Response
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb..."


In [20]:
def generate_questions(sample_df, num_questions=3):
    all_questions = []

    for index, row in sample_df.iterrows():
        context = row['Context']
        response = row['Response']
        
        # Prepare the prompt
        prompt = f"""
        You are emulating a person experiencing mental health challenges.
        Formulate {num_questions} insightful questions based on the following mental health conversation record. Use the context provided to generate relevant questions, but avoid directly copying words from the response.

        Record:
        Context: {context}
        Response: {response}

        Provide the output in a format that can be easily parsed from a dataframe:
        "Question 1", "Question 2", "Question 3"
        """
        
        # Generate the questions using the pipeline
        result = pipe(prompt, **generation_args)[0]["generated_text"]
        
        # Extract questions from the generated text
        questions = []
        for line in result.split('\n'):
            if 'Question' in line:
                parts = line.split(":", 1)
                if len(parts) > 1:
                    question_text = parts[1].strip()
                    if question_text:
                        questions.append(question_text)
        
        # Ensure we have exactly num_questions, regenerate if needed
        while len(questions) < num_questions:
            remaining_questions = num_questions - len(questions)
            prompt = f"""
            Generate {remaining_questions} more questions based on the following context:

            Context: {context}
            Response: {response}

            Provide the output in the following format:
            Question {len(questions) + 1}: [Your question here]
            """
            result = pipe(prompt, **generation_args)[0]["generated_text"]
            for line in result.split('\n'):
                question_num = len(questions) + 1
                if f'Question {question_num}' in line:
                    parts = line.split(":", 1)
                    if len(parts) > 1:
                        question_text = parts[1].strip()
                        if question_text:
                            questions.append(question_text)
        
        # Ensure we only add the required number of questions
        all_questions.append({
            'Context': context,
            'Questions': questions[:num_questions]
        })
    
    return all_questions

# Example usage with DataFrame
sample_df = df1.sample(n=3, random_state=42)

questions_list = generate_questions(sample_df)
for record in questions_list:
    print(f"Context: {record['Context']}")
    for i, question in enumerate(record['Questions'], 1):
        print(f"Question {i}: {question}")
    print()


KeyboardInterrupt: 

In [23]:
def generate_questions(sample_df, num_questions=3):
    all_questions = []

    for index, row in sample_df.iterrows():
        context = row['Context']
        response = row['Response']
        
        # Prepare the prompt to generate all questions in one go
        prompt = f"""
        You are emulating a person experiencing mental health challenges.
        Formulate {num_questions} insightful questions based on the following mental health conversation record. Use the context provided to generate relevant questions, but avoid directly copying words from the response.

        Record:
        Context: {context}
        Response: {response}

        Provide the output in a format that can be easily parsed from a dataframe:
        "Question 1", "Question 2", "Question 3"
        """

        # Generate the questions using the pipeline
        result = pipe(prompt, **generation_args)[0]["generated_text"]

        # Extract questions from the generated text
        questions = []
        for line in result.split('\n'):
            if 'Question' in line:
                parts = line.split(":", 1)
                if len(parts) > 1:
                    question_text = parts[1].strip()
                    if question_text:
                        questions.append(question_text)
        
        # Ensure we have exactly num_questions
        while len(questions) < num_questions:
            remaining_questions = num_questions - len(questions)
            prompt = f"""
            Generate {remaining_questions} more questions based on the following context:

            Context: {context}
            Response: {response}

            Provide the output in the following format:
            Question {len(questions) + 1}: [Your question here]
            """
            result = pipe(prompt, **generation_args)[0]["generated_text"]
            for line in result.split('\n'):
                question_num = len(questions) + 1
                if f'Question {question_num}' in line:
                    parts = line.split(":", 1)
                    if len(parts) > 1:
                        question_text = parts[1].strip()
                        if question_text:
                            questions.append(question_text)
        
        # Add the questions to the list with their corresponding context
        all_questions.append({
            'Context': context,
            'Questions': questions
        })
    
    return all_questions

# Example usage with DataFrame
sample_df = df1.sample(n=3, random_state=42)

questions_list = generate_questions(sample_df)
for record in questions_list:
    print(f"Context: {record['Context']}")
    for i, question in enumerate(record['Questions'], 1):
        print(f"Question {i}: {question}")
    print()


Context: I've hit my head on walls and floors ever since I was young. I sometimes still do it but I don't exactly know why,    I have anxiety and I had a rough childhood but now I'll start to hit my head and sometimes not realize it but I don't know how to stop or even why I'm doing it.    How can I help myself to change my behavior?
Question 1: What specific triggers or situations make you more likely to hit your head, and how can you identify these patterns?",
Question 2: Can you describe the thoughts or emotions that precede the act of hitting your head, and how might these be addressed in therapy?",
Question 3: What positive behaviors or activities have you found to be helpful in managing your anxiety and preventing the urge to hit your head?"
Question 4: In what ways have your past experiences, particularly your childhood, shaped your current coping mechanisms, and how can we work together to develop healthier strategies for dealing with stress and anxiety?",
Question 5: How do yo

In [24]:
def score_questions(questions, context):
    # Dummy scoring function; you can replace this with a more sophisticated method
    scores = []
    for question in questions:
        # Simple scoring: length of question relative to context length
        score = len(question) / len(context)
        scores.append(score)
    return scores

def generate_questions(sample_df, num_questions=3):
    all_questions = []

    for index, row in sample_df.iterrows():
        context = row['Context']
        response = row['Response']
        
        # Prepare the prompt to generate all questions in one go
        prompt = f"""
        You are emulating a person experiencing mental health challenges.
        Formulate {num_questions*2} insightful questions based on the following mental health conversation record. Use the context provided to generate relevant questions, but avoid directly copying words from the response.

        Record:
        Context: {context}
        Response: {response}

        Provide the output in a format that can be easily parsed from a dataframe:
        "Question 1", "Question 2", "Question 3", ...
        """

        # Generate the questions using the pipeline
        result = pipe(prompt, **generation_args)[0]["generated_text"]

        # Extract questions from the generated text
        questions = []
        for line in result.split('\n'):
            if 'Question' in line:
                parts = line.split(":", 1)
                if len(parts) > 1:
                    question_text = parts[1].strip()
                    if question_text:
                        questions.append(question_text)
        
        # Score and select top num_questions questions
        if len(questions) > num_questions:
            scores = score_questions(questions, context)
            sorted_questions = [q for _, q in sorted(zip(scores, questions), reverse=True)]
            selected_questions = sorted_questions[:num_questions]
        else:
            selected_questions = questions
        
        # Add the questions to the list with their corresponding context
        all_questions.append({
            'Context': context,
            'Questions': selected_questions
        })
    
    return all_questions

# Example usage with DataFrame
sample_df = df1.sample(n=3, random_state=42)

questions_list = generate_questions(sample_df)
for record in questions_list:
    print(f"Context: {record['Context']}")
    for i, question in enumerate(record['Questions'], 1):
        print(f"Question {i}: {question}")
    print()

Context: I've hit my head on walls and floors ever since I was young. I sometimes still do it but I don't exactly know why,    I have anxiety and I had a rough childhood but now I'll start to hit my head and sometimes not realize it but I don't know how to stop or even why I'm doing it.    How can I help myself to change my behavior?
Question 1: "What specific type of therapy has been proven helpful for depression and anxiety?", "Question 2": "What is the role of medication in managing severe anxiety?", "Question 3": "Why is it not recommended to try to manage severe anxiety without professional help?", "Question 4": "What is the difference between Cognitive Behavioral Therapy and trauma informed therapy?", "Question 5": "How can one learn to recognize and deal with the underlying cause of a harmful behavior?", "Question 6": "What are some positive behaviors that can replace the harmful behavior of hitting one's head?"}
Question 2: What are some positive behaviors that can replace the 

In [26]:
def score_questions(questions, context):
    # Improved scoring function; can be customized
    scores = []
    for question in questions:
        # Basic scoring: consider length and relevance
        score = len(question) / len(context)
        scores.append(score)
    return scores

def generate_questions(sample_df, num_questions=3):
    all_questions = []

    for _, row in sample_df.iterrows():
        context = row['Context']
        response = row['Response']
        
        # Prepare the prompt
        prompt = f"""
        You are emulating a person experiencing mental health challenges.
        Formulate {num_questions*2} insightful questions based on the following mental health conversation record. 
        Use the context provided to generate relevant questions, but avoid directly copying words from the response.

        Record:
        Context: {context}
        Response: {response}

        Provide the output in the format:
        "Question 1: Question text"
        "Question 2: Question text"
        ...
        """

        # Generate the questions
        result = pipe(prompt, **generation_args)[0]["generated_text"]

        # Extract questions
        questions = []
        for line in result.split('\n'):
            if 'Question' in line:
                parts = line.split(":", 1)
                if len(parts) > 1:
                    question_text = parts[1].strip()
                    if question_text:
                        questions.append(question_text)
        
        # Score and select top num_questions questions
        if len(questions) > num_questions:
            scores = score_questions(questions, context)
            sorted_questions = [q for _, q in sorted(zip(scores, questions), reverse=True)]
            selected_questions = sorted_questions[:num_questions]
        else:
            selected_questions = questions
        
        # Add the questions to the list with their corresponding context
        all_questions.append({
            'Context': context,
            'Questions': selected_questions
        })
    
    return all_questions

# Example usage with DataFrame
sample_df = df1.sample(n=3, random_state=42)

questions_list = generate_questions(sample_df)
for record in questions_list:
    print(f"Context: {record['Context']}")
    for i, question in enumerate(record['Questions'], 1):
        print(f"Question {i}: {question}")
    print()

Context: I've hit my head on walls and floors ever since I was young. I sometimes still do it but I don't exactly know why,    I have anxiety and I had a rough childhood but now I'll start to hit my head and sometimes not realize it but I don't know how to stop or even why I'm doing it.    How can I help myself to change my behavior?
Question 1: In what ways do you feel supported by your friends, family, or others in your life, and how do you think you could open up about your struggles to them?"
Question 2: Are there certain times of day or specific situations where you feel more inclined to hit your head, and if so, what might be triggering those moments?"
Question 3: Have you noticed any patterns or recurring themes in your life that might be connected to the anxiety and self-harming behaviors you've experienced?"

Context: Over a year ago I had a female friend. She turned out to be kind of crazy so I decided to stop talking to her. When she would call me I wouldn't answer the phone

In [29]:
from collections import defaultdict

def generate_questions(sample_df, num_questions=2):
    all_questions = []

    for _, row in sample_df.iterrows():
        context = row['Context']
        response = row['Response']
        
        # Prepare the prompt
        prompt = f"""
        You are emulating a person experiencing mental health challenges.
        Formulate {num_questions*2} insightful questions based on the following mental health conversation record. 
        Use the context provided to generate relevant questions, but avoid directly copying words from the response.

        Record:
        Context: {context}
        Response: {response}

        Provide the output in the format:
        "Question 1: Question text"
        "Question 2: Question text"
        ...
        """

        # Generate the questions
        result = pipe(prompt, **generation_args)[0]["generated_text"]

        # Extract questions
        questions = []
        for line in result.split('\n'):
            if 'Question' in line:
                parts = line.split(":", 1)
                if len(parts) > 1:
                    question_text = parts[1].strip()
                    if question_text:
                        questions.append(question_text)
        
        # Score and select top num_questions questions
        if len(questions) > num_questions:
            scores = score_questions(questions, context)
            sorted_questions = [q for _, q in sorted(zip(scores, questions), reverse=True)]
            selected_questions = sorted_questions[:num_questions]
        else:
            selected_questions = list(dict.fromkeys(questions))  # Remove duplicates if less than num_questions
        
        # Add the questions to the list with their corresponding context
        all_questions.append({
            'Context': context,
            'Questions': selected_questions
        })
    
    return all_questions

# Example usage with DataFrame
sample_df = df1.sample(n=500, random_state=42)

# questions_list = generate_questions(sample_df)
# for record in questions_list:
#     print(f"Context: {record['Context']}")
#     for i, question in enumerate(record['Questions'], 1):
#         print(f"Question {i}: {question}")
#     print()

Context: I've hit my head on walls and floors ever since I was young. I sometimes still do it but I don't exactly know why,    I have anxiety and I had a rough childhood but now I'll start to hit my head and sometimes not realize it but I don't know how to stop or even why I'm doing it.    How can I help myself to change my behavior?
Question 1: What resources or support systems are available for someone like me, who is recognizing their mental health needs but uncertain about how to proceed with changing harmful habits?"
Question 2: Given that my childhood experiences might be influencing my current behavior, how can I seek therapy that acknowledges and treats trauma differently than general anxiety therapy?"

Context: Over a year ago I had a female friend. She turned out to be kind of crazy so I decided to stop talking to her. When she would call me I wouldn't answer the phone. This made my girlfriend really suspicious. She would ask me why I wouldn't ever answer that phone number. I

In [30]:
questions_list = generate_questions(sample_df)
# for record in questions_list:
#     print(f"Context: {record['Context']}")
#     for i, question in enumerate(record['Questions'], 1):
#         print(f"Question {i}: {question}")
#     print()

In [31]:
from collections import defaultdict

def generate_questions(sample_df, num_questions=2):
    all_questions = []

    for _, row in sample_df.iterrows():
        context = row['Context']
        response = row['Response']
        
        # Prepare the prompt
        prompt = f"""
        You are emulating a person experiencing mental health challenges.
        Formulate {num_questions*2} insightful questions based on the following mental health conversation record. 
        Use the context provided to generate relevant questions, but avoid directly copying words from the response.

        Record:
        Context: {context}
        Response: {response}

        Provide the output in the format:
        "Question 1: Question text"
        "Question 2: Question text"
        ...
        """

        # Generate the questions
        result = pipe(prompt, **generation_args)[0]["generated_text"]

        # Extract questions
        questions = []
        for line in result.split('\n'):
            if 'Question' in line:
                parts = line.split(":", 1)
                if len(parts) > 1:
                    question_text = parts[1].strip()
                    if question_text:
                        questions.append(question_text)
        
        # Score and select top num_questions questions
        if len(questions) > num_questions:
            scores = score_questions(questions, context)
            sorted_questions = [q for _, q in sorted(zip(scores, questions), reverse=True)]
            selected_questions = sorted_questions[:num_questions]
        else:
            selected_questions = list(dict.fromkeys(questions))  # Remove duplicates if less than num_questions
        
        # Add the questions to the list with their corresponding context
        all_questions.append({
            'Context': context,
            'Questions': selected_questions
        })
    
    return all_questions

# Example usage with DataFrame
sample_df = df1.sample(n=500, random_state=42)

# questions_list = generate_questions(sample_df)
# for record in questions_list:
#     print(f"Context: {record['Context']}")
#     for i, question in enumerate(record['Questions'], 1):
#         print(f"Question {i}: {question}")
#     print()

In [32]:
questions_list = generate_questions(sample_df)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [33]:
# Print the first few records to inspect the output
for i, record in enumerate(questions_list[:5]):  # Adjust the number as needed
    print(f"Context {i + 1}: {record['Context']}")
    for j, question in enumerate(record['Questions'], 1):
        print(f"  Question {j}: {question}")
    print()  # For spacing between different records


Context 1: I've hit my head on walls and floors ever since I was young. I sometimes still do it but I don't exactly know why,    I have anxiety and I had a rough childhood but now I'll start to hit my head and sometimes not realize it but I don't know how to stop or even why I'm doing it.    How can I help myself to change my behavior?

Context 2: Over a year ago I had a female friend. She turned out to be kind of crazy so I decided to stop talking to her. When she would call me I wouldn't answer the phone. This made my girlfriend really suspicious. She would ask me why I wouldn't ever answer that phone number. I told my girlfriend that I don't want to be friends with that other woman, but I don't think she believes me. How can I get my girlfriend to understand?

Context 3: My long-distance girlfriend is in a sorority, and it's changing her. I feel like I'm becoming less important to her and it hurts. She just wants me to support the sorority, but it's so hard. I try every day to show 

In [35]:
import copy

# Assuming questions_list is your original list
questions_list_copy = copy.deepcopy(questions_list)

# Now you can apply the ID-assignment function to the copied list
# questions_list_with_ids = add_ids_to_questions_list(questions_list_copy)

# The original questions_list remains unchanged, and you can inspect questions_list_with_ids


In [34]:
# Calculate the total number of questions generated
total_questions = sum(len(record['Questions']) for record in questions_list)
print(f"Total number of generated questions: {total_questions}")


Total number of generated questions: 860


In [36]:
import uuid

def add_ids_to_questions_list(questions_list_copy):
    # Initialize a new list to store results with IDs
    questions_with_ids = []

    # Iterate through the questions list and assign unique IDs
    for record in questions_list_copy:
        context = record['Context']
        questions = record['Questions']
        
        # Generate a unique ID for this context and associated questions
        unique_id = uuid.uuid4().hex[:10]  # Generate a shorter unique ID
        
        # Add ID to each record and the questions
        questions_with_ids.append({
            'ID': unique_id,
            'Context': context,
            'Questions': [{'ID': unique_id, 'Question': question} for question in questions]
        })

    return questions_with_ids

# Apply the ID assignment function
questions_list_with_ids = add_ids_to_questions_list(questions_list_copy)

# Example output inspection
for record in questions_list_with_ids[:5]:  # Adjust the number to view more records
    print(f"ID: {record['ID']}")
    print(f"Context: {record['Context']}")
    for question in record['Questions']:
        print(f"  QuestionID: {question['ID']}, Question: {question['Question']}")
    print()  # For spacing between records


ID: 82b18f1d35
Context: I've hit my head on walls and floors ever since I was young. I sometimes still do it but I don't exactly know why,    I have anxiety and I had a rough childhood but now I'll start to hit my head and sometimes not realize it but I don't know how to stop or even why I'm doing it.    How can I help myself to change my behavior?

ID: b9ea4fc5fa
Context: Over a year ago I had a female friend. She turned out to be kind of crazy so I decided to stop talking to her. When she would call me I wouldn't answer the phone. This made my girlfriend really suspicious. She would ask me why I wouldn't ever answer that phone number. I told my girlfriend that I don't want to be friends with that other woman, but I don't think she believes me. How can I get my girlfriend to understand?

ID: f307b325d9
Context: My long-distance girlfriend is in a sorority, and it's changing her. I feel like I'm becoming less important to her and it hurts. She just wants me to support the sorority, but

In [37]:
import pandas as pd

# Flatten the questions_list_with_ids to create the DataFrame
def flatten_questions(questions_list_with_ids):
    # Initialize lists to hold data for the DataFrame
    question_texts = []
    document_ids = []
    
    for record in questions_list_with_ids:
        unique_id = record['ID']
        for question in record['Questions']:
            question_texts.append(question['Question'])
            document_ids.append(unique_id)
    
    # Create the DataFrame
    df2 = pd.DataFrame({
        'questions': question_texts,
        'documents': document_ids
    })
    
    return df2

# Apply the flattening function
questions_df = flatten_questions(questions_list_with_ids)

In [38]:
print(questions_df.shape)  # Should be (860, 2)

(860, 2)


In [40]:
questions_df.head(10)

,questions,documents
0,How do you both feel about seeking external su...,f307b325d9
1,Have you previously used any form of communica...,f307b325d9
2,What steps can you take to ensure that your ow...,842e439c52
3,How can setting personal boundaries and practi...,842e439c52
4,In what ways have your past experiences shaped...,af8cba43be
5,What are some moments or activities that bring...,af8cba43be
6,"Looking forward, what qualities do you hope to...",8c303a7fdc
7,Reflecting on your own feelings during the bre...,8c303a7fdc
8,"Considering the influence of cognitive biases,...",b1023f460a
9,Reflecting on the role of emotional intelligen...,b1023f460a


In [41]:
# Save the DataFrame to a CSV file
questions_df.to_csv('ground_truth_data.csv', index=False)


OSError: [Errno 28] No space left on device

In [42]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         100G   43G   58G  43% /
tmpfs            64M     0   64M   0% /dev
tmpfs           7.7G     0  7.7G   0% /sys/fs/cgroup
/dev/nvme0n1p1  100G   43G   58G  43% /run
tmpfs            14G  4.0K   14G   1% /dev/shm
/dev/nvme2n1    2.0G  1.9G     0 100% /home/jovyan
tmpfs            14G  120K   14G   1% /home/jovyan/.saturn
tmpfs            14G   12K   14G   1% /run/secrets/kubernetes.io/serviceaccount
tmpfs           7.7G   12K  7.7G   1% /proc/driver/nvidia
tmpfs           7.7G   19M  7.7G   1% /run/nvidia-persistenced/socket
tmpfs           7.7G     0  7.7G   0% /proc/acpi
tmpfs           7.7G     0  7.7G   0% /sys/firmware


In [45]:
# Move the large incomplete Hugging Face cache file to /run/cache/
!mv /home/jovyan/.cache/huggingface/hub/models--microsoft--Phi-3-mini-128k-instruct/blobs/fc534bed68dc12e48d1f28d144bfb2239bd1ef4f5d5892d68e9d1f85731b8c12.incomplete /run/cache/

# Move the entire Hugging Face cache directory for the specific model to /run/cache/
!mv /home/jovyan/.cache/huggingface/hub/models--microsoft--Phi-3-mini-128k-instruct /run/cache/

# Move the pip cache directory to /run/cache/
!mv /home/jovyan/.cache/pip /run/cache/


In [46]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         100G   45G   56G  45% /
tmpfs            64M     0   64M   0% /dev
tmpfs           7.7G     0  7.7G   0% /sys/fs/cgroup
/dev/nvme0n1p1  100G   45G   56G  45% /run
tmpfs            14G  4.0K   14G   1% /dev/shm
/dev/nvme2n1    2.0G  9.9M  1.9G   1% /home/jovyan
tmpfs            14G  120K   14G   1% /home/jovyan/.saturn
tmpfs            14G   12K   14G   1% /run/secrets/kubernetes.io/serviceaccount
tmpfs           7.7G   12K  7.7G   1% /proc/driver/nvidia
tmpfs           7.7G   20M  7.7G   1% /run/nvidia-persistenced/socket
tmpfs           7.7G     0  7.7G   0% /proc/acpi
tmpfs           7.7G     0  7.7G   0% /sys/firmware


In [47]:
import pickle
questions_df.to_pickle('questions_documents.pkl')